# Workshop 4: Training and Evaluation

In [20]:
import numpy as np
import random

def get_data(path):
    f = open(path, 'r')
    
    lines = f.readlines()
    
    training_images = np.zeros((len(lines), 784))
    training_labels = np.zeros((len(lines), 10))
    index = 0
    for line in lines:
        line = line.strip()
        label = int(line[0])
        training_images[index, :] = np.fromstring(line[2:], dtype=int, sep=',')
        training_labels[index, label - 1] = 1.0
        index += 1
        

    f.close()
    
    return training_images / 255, training_labels

def sigmoid(x):
    return np.where(x >= 0, 
                    1 / (1 + np.exp(-x)), 
                    np.exp(x) / (1 + np.exp(x)))

Last week, we implemented backpropagation and gradient descent for our neural network. This week, we will finally put everything together and train our network to read hand-written digits.

## Training / Validation

If you recall, we will need to show our network many different examples of numbers in order for it to learn effectively. The images we will show our network is known as the *training set*. We will also want some way to evaluate the performance of our network after it learns. For this, we will need a seperate set of data that the network has never seen before. This is the *validation set*. The training set should be larger than the validation set. In our case, there are 60000 image, label pairs in the training set, and 10000 image, label pairs in the validation set.

The training process is simple. For every image in the training set, forward pass the image into the network, then backpropagate and update the weights. When we make it through the entire training set, we say that we have completed one *epoch*. Typically, we will need to train for several epochs -- that is to say, we often need to run through the training set multiple times.

We also want to evaluate our network. For every image in the validation set, we will forward pass the image into the network. Then, we will look at the output, a vector of length 10 -- our network network is trained to place a 1 in the correct entry of the vector, and a 0 in all other entries. Therefore, the maximal entry in the network's output is the number that the network thinks the given image represents. We then compare the network's answer to the correct answer, the given label. After observing every image in the validation set, we note the proportion of the samples that the network got correct.

Sometimes, it can be helpful to run through the validation set after every epoch, to see how the network's performance improves over time. This can be helpful for deciding how many epochs you want to run your network for.

Also, in order to train our network effectively, we will need to pick an appropriate learning rate. If the learning rate is too large, the network will take too large steps during the gradient descent, and it will be ineffective and unstable. If the learning rate is too small, the training could be slow. We will find a good learning rate through trial and error. Remember, the learning rate should be a very small positive number, typically less than 1.

Try to write your own training and validation methods. The code for forward pass and backpropagation is provided for you. See if you can get a validation accuracy of over 90 percent.

In [31]:
class NeuralNetwork():
    """
    A Fully Connected Neural Network. There are 784 input layer nodes, 12 hidden layer nodes, and 10 output layer
    nodes.
    """
    def __init__(self):
        
        
        # Arrays to hold node values
        self.N = np.zeros((784, ))
        self.H = np.zeros((12, ))
        self.Z = np.zeros((10, ))
        
        # Arrays to hold weight values (randomly initialized between -1 and 1)
        self.W = 2 * np.random.rand(784, 12) - 1
        self.V = 2 * np.random.rand(12, 10) - 1
        
        # Arrays to hold biases for hidden and output nodes
        self.B = 2 * np.random.rand(12) - 1
        self.C = 2 * np.random.rand(10) - 1
        
        
        # Arrays to store values for calculating backprop efficiently
        self.H_before = np.zeros((12, ))
        self.Z_before = np.zeros((10, ))
        
        self.W_grad = np.zeros((784, 12))
        self.V_grad = np.zeros((12, 10))
        self.B_grad = np.zeros((12, ))
        self.C_grad = np.zeros((10, ))
        
        
    def forward(self, x):
        self.N = x
        self.H_before = np.dot(self.N, self.W) + self.B
        self.H = np.tanh(self.H_before)
        self.Z_before = np.dot(self.H, self.V) + self.C
        self.Z = 1 / (1 + np.exp(-1 * self.Z_before))

        
    def calculate_loss(self, x, y):
        out = self.forward(x)
        loss = np.sum((self.Z - y) ** 2)
        return loss

        
    def backpropagate(self, label):
        dZ = -2 * (label - self.Z)
        dZ_before = dZ * sigmoid(self.Z_before) * (1 - sigmoid(self.Z_before))
        self.V_grad = np.outer(self.H, dZ_before)
        self.C_grad = dZ_before
        
        dH = np.dot(dZ_before, np.transpose(self.V))
        dH_before = dH * (1 / (np.cosh(self.H_before) ** 2))
        self.W_grad = np.outer(self.N, dH_before)
        self.B_grad = dH_before
        
    
    def update(self, lr):
        self.V -= lr * self.V_grad
        self.C -= lr * self.C_grad
        self.W -= lr * self.W_grad
        self.B -= lr * self.B_grad
        
        
    def train(self, training_images, training_labels, val_images, val_labels):
        """
        Write a training method for the neural network. Add whatever method parameters you think you might need.
        """
        
        lr = 0.05
        epoch = 6
        
        
        for epoch in range(epoch):
            print("Epoch: ", epoch + 1)
            losses = []
            for i, image in enumerate(training_images):
                self.forward(image)
                loss = self.calculate_loss(image, training_labels[i])
                losses.append(loss)
                self.backpropagate(training_labels[i])
                self.update(lr)
                
                if i % 1000 == 0 and i != 0:
                    print(sum(losses) / len(losses))
                
            acc = self.evaluate( val_images, val_labels)
            print("accuracy", acc)
                
            
    def evaluate(self, val_images, val_labels):
        """
        Write an evaluation method for the neural network. Add whatever method parameters you think you might need.
        """
        correct = 0
        for i, image in enumerate(val_images):
            self.forward(image)
            
            #[ 0.2, 0.3, 0.4, 0.01, 0.001, .... ]
        
            pred = np.argmax(self.Z)
            
            #[0,0,1,0,0,....]
            # val labels [..., ..., ...]
            if val_labels[i][pred] == 1:
                correct += 1
        
        
        acc = correct / len(val_images)
        
        return acc
            
            
            
            
            
        
            


Test your code below

In [24]:
training_images, training_labels = get_data("../data/mnist_train.csv")
val_images, val_labels = get_data("../data/mnist_test.csv")

In [32]:
model = NeuralNetwork()
model.train(training_images, training_labels, val_images, val_labels)

Epoch:  1
0.9686231204433425
0.8440319268976434
0.7640840494324986
0.6990697650718135
0.6499315921806432
0.6076694183791174
0.5726193577234765
0.5492813169450509
0.5290145153321166
0.5060728655759159
0.4878529935891601
0.4740018661830022
0.4627111672899104
0.4531144295699307
0.44831456078056114
0.4378727852158407
0.4293203926923888
0.42272686738387116
0.4127116669512751
0.4058589688611139
0.39920178562377273
0.39058984532257546
0.3860762907585559
0.38062317952743785
0.37547747614362464
0.37037592845711353
0.36607538793528616
0.3619783123764603
0.35721737531285325
0.35490036553806026
0.35270474712061667
0.35002609914732136
0.3483424567924917
0.34454725492085825
0.34191603966910594
0.33834011653034124
0.3352619707058135
0.333620232186784
0.3312406946252502
0.3296436260103459
0.3269126977957582
0.32493043609151745
0.3235022229088621
0.32048973302419265
0.31817523208473397
0.3166931563547131
0.3149744425208379
0.31380910085058117
0.3117929998680092
0.31133024418993993
0.31009003979166927
0

## Challenge

After succesfully training, you may realize that our network has a difficult time getting accuracies higher than low nineties. From the testing I've done, that seems to be the best our basic network can do -- however, it has been shown that neural networks are capable of reaching accuracies over 99 percent for the MNIST dataset. What changes can we make to the structure of our network to accomplish this? More nodes in the hidden layer? Add another hidden layer? Perhaps there are even more sophisticated methods...

Try to create a neural network that can achieve a validation accuracy of over 96 percent. It's probably easiest if you reuse code from the previous network. Feel free to try any method you can think of or find on the internet.